# Classification and Regression with a Neural Network in Pytorch

Welcome to your first programming assignment! 
In this assignment you will build your first classification and regression neural networks.
This assignment is a 'step-through' guide to implement a simple fully-connected neural network in Pytorch.

* In the first part of this exercise, you will implement a neural network with a 2 dimensional input. Your dataset is based on a two circles-shaped groups for classification.

* Then, in the second part of this exercise, you will implement a regression model for predicting the output of a two dimensional function.

# Packages
Let's first import all the packages that you will need during this part of assignment.

Feel free to use another libraries if you want to.

In [37]:
import numpy as np
from sklearn.datasets import make_moons, make_circles
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_curve, roc_auc_score, auc
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt
import torch
from torch import nn

# Dataset

In [38]:
random_num = 0
np.random.seed(random_num)
torch.manual_seed(random_num)
x, y = make_circles(500, noise=0.075)
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3)

**Visualize the dataset using matplotlib:**

In [43]:
### START CODE HERE ###

# Simple Logistic Regression

1. Implement a 2-class classification neural network with a zero hidden layer.
2. Plot loss vs epoch.
3. Plot AUC vs epoch for train and test sets. 
4. Plot ROC curve and calculate AUC for the test set.
5. Plot the learned decision boundary.
6. Briefly interpret graph's results.   

How you will do it:

* Prepare the Data.
* Define the Model.
* Train the Model.
* Evaluate the Model.
* Visualization.

**Define the Model:**

In [158]:
### START CODE HERE ###

In [151]:
def initialize_param(dim):
    w=np.zeros((dim,1))#creates a vector of zeros of shape (dim, 1) for w
    return w

In [153]:
dim = 2
w= initialize_param(dim)
print ("w = " + str(w))

w = [[0.]
 [0.]]


In [154]:
def sigmoid(z):
    s = 1/(1 + np.exp(-z))
    return s

In [155]:
print ("sigmoid([0, 2]) = " + str(sigmoid(np.array([0,2]))))

sigmoid([0, 2]) = [0.5        0.88079708]


In [159]:
def forward(x,w,y): #computing loss function
    activation=sigmoid(np.dot(w.T,x))
    loss= -np.mean(y*np.log(activation) + (1-y)*np.log(1-activation)) #negative log-likelihood loss function for LR
    return loss, activation

In [162]:
w,X, Y = np.array([[1],[2]]), np.array([[1,2],[3,4]]), np.array([[1,0]])
cost, A = forward(w, X, Y)
print ("cost = " + str(cost))
print(A)

cost = 4.25047843267675
[[0.99908895]
 [0.9999546 ]]


In [163]:
def backpropagation(x,y,activation):
    grad=np.mean(np.dot(x, (activation-y).T))
    return grad

In [167]:
grad = backpropagation(X, Y, A)
print(grad)

2.9976088773422425


In [179]:
def updateW(w, x, y, num_iterations, learning_rate):
    loss =[]
    for i in range(num_iterations):
        loss,a= forward(x,w,y)
        gradient= backpropagation(x,y,a)

        #update
        w=w-learning_rate*gradient
        ##print(loss,A,gradient,w)
    return w

In [176]:
w=updateW(w, X, Y, num_iterations=100, learning_rate=0.1)
print (w)

[[-0.77153922]
 [ 0.22846078]]


In [185]:
def predict(x,w):
    n,m=X.shape[0],x.shape[1]
    w = w.reshape(n, 1)
    activation=sigmoid(np.dot(w.T,x))

    y_pred=np.zeros(m)
    for i in range(m):
        if activation[0,i] >0.5:y_pred[i]=1
        else: y_pred[i]=0
    return y_pred

In [186]:
print ("predictions = " + str(predict(X,w)))

predictions = [0. 0.]


**Training and validation:**

In [23]:
### START CODE HERE ###


**Visualizing the plots**:

In [24]:
### START CODE HERE ###


7. Is the decision boundary linear or nonlinear in the case of a logistic regression? Explain.

# Classification with Neural Networks

After you implemented "classical" logistic regression, now you will be implementing a neural network with one or more hidden layers.
You will need to choose the number of hidden layers and nodes in a feedforward neural network, activation function, the type of optimizer and its hyperparmeters which will give you the best result. Remember, we don't want to overfit the training data, we want to generalize the solution for new data not seen during training. 

Plot the same graphs as in the previous sections and explain the similarities and differences. 

**Define the Model:**

In [25]:
### START CODE HERE ###


**Training and validation:**

In [26]:
### START CODE HERE ###


**Visualizing the plots:**

In [27]:
### START CODE HERE ###


8. Why does a neural network need a non-linear activation function? try to figure out what heppan when you remove the activation function and explain the results.

# Regression with Neural Networks

In this part of the excercise you will need to implement a regression model using neural networks. The model should predict the output of a trigonometric function of two variables. Your data set is based on a meshgrid. Your task is to create a list of points that would correspond to a grid and use it for the input of your neural network. Then, build your neural networks and find the architecture which gives you the best results.
1. Plot the surface from the overall data and compare it to your predicted test sets.
2. Which loss function and validation metric did you choose?
3. Plot the loss and validation metrics vs epoch for the training and test sets.
4. Build a new neural network and try overfitting your training set. Show the overfitting by using learning curve plots. 
    **Note**: You can use plt.ylim() function to better focus on the changes in the trends.

# Packages 

First import all the packages that you will need during this part of assignment.

Feel free to use another libraries if you want to.

In [28]:
import numpy as np
import torch
from torch.autograd import Variable
from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt
from matplotlib import cm

**Generate data:**

In [29]:
np.random.seed(random_num)
x = np.linspace(-5, 5, 30)
y = np.linspace(-5, 5, 30)
xx, yy = np.meshgrid(x, y)
z = np.sin(xx) * np.cos(yy) + 0.1 * np.random.rand(xx.shape[0], xx.shape[1])

**Define the Model:**

In [30]:
### START CODE HERE ###

**Training and validation:**

In [31]:
### START CODE HERE ###


**Visualizing the plots:**

In [32]:
### START CODE HERE ###


### Build a new neural network and try overfitting your training set

**Generate data:**

In [33]:
np.random.seed(random_num)
x = np.linspace(-5, 5, 30)
y = np.linspace(-5, 5, 30)
xx, yy = np.meshgrid(x, y)
z = np.sin(xx) * np.cos(yy) + 0.1 * np.random.rand(xx.shape[0], xx.shape[1])

**Define the Model:**

In [34]:
### START CODE HERE ###


**Training and validation:**

In [35]:
### START CODE HERE ###


**Visualizing the plots:**

In [36]:
### START CODE HERE ###


5. Briefly explain graph's results.

6. How does your metric value differs between the training data and the test data and why?
